In [23]:
!pip install python-docx


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
# Paso 1: Importar bibliotecas necesarias
import pandas as pd
from docx import Document
import os

In [25]:
# Paso 2: Definir rutas de los archivos
word_file = "Corpus_extranjerIA.docx"  # Nombre del archivo de Word
excel_file = "corpus_extranjerIA.xlsx"  # Nombre del archivo de Excel de destino



In [26]:
# # Paso 3: Función para leer el archivo de Word y extraer datos
# def extract_questions_answers(word_path):
#     doc = Document(word_path)
#     data = []
#     current_document = ""
#     current_route = ""
#     current_category = ""
    
#     paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    
#     for i in range(len(paragraphs)):
#         text = paragraphs[i]
        
#         if text.startswith("Archivo:"):
#             current_document = text.replace("Archivo:", "").strip()
#         elif text.startswith("Ruta:"):
#             current_route = text.replace("Ruta:", "").strip()
#         elif text.startswith("Categoría:"):
#             current_category = text.replace("Categoría:", "").strip()
#         elif text.startswith("Pregunta:"):
#             question = text.replace("Pregunta:", "").strip()
#             answer = ""
            
#             if "Respuesta:" in question:
#                 question, answer = question.split("Respuesta:", 1)
#                 question = question.strip()
#                 answer = answer.strip()
#             elif i + 1 < len(paragraphs) and paragraphs[i + 1].startswith("Respuesta:"):
#                 answer = paragraphs[i + 1].replace("Respuesta:", "").strip()
                
#             data.append([current_document, current_route, current_category, question, answer])
    
#     return pd.DataFrame(data, columns=["Documento", "Ruta", "Categoria_pregunta", "Pregunta", "Respuesta_Humana"])

In [27]:
# Paso 3: Función para leer el archivo de Word y extraer datos
def extract_questions_answers(word_path):
    doc = Document(word_path)
    data = []
    current_document = ""
    current_route = ""
    current_category = ""
    
    paragraphs = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
    
    for i in range(len(paragraphs)):
        text = paragraphs[i]
        
        if text.startswith("Archivo:"):
            current_document = text.replace("Archivo:", "").strip()
        elif text.startswith("Ruta:"):
            current_route = text.replace("Ruta:", "").strip()
        elif text.startswith("Categoría:"):
            current_category = text.replace("Categoría:", "").strip()
        elif text.startswith("Pregunta:"):
            question = text.replace("Pregunta:", "").strip()
            answer = ""
            
            if "Respuesta:" in question:
                question, answer = question.split("Respuesta:", 1)
                question = question.strip()
                answer = answer.strip()
            elif i + 1 < len(paragraphs) and paragraphs[i + 1].startswith("Respuesta:"):
                # Capturar respuesta inicial
                answer = paragraphs[i + 1].replace("Respuesta:", "").strip()
                
                # Capturar respuestas multilínea (listas de pasos, documentos, etc.)
                for j in range(i + 2, len(paragraphs)):
                    next_text = paragraphs[j]
                    
                    # Si encontramos una nueva pregunta o categoría, detener captura
                    if next_text.startswith("Pregunta:") or next_text.startswith("Categoría:"):
                        break
                    
                    # Si es parte de la respuesta, agregarla con un salto de línea
                    answer += "\n" + next_text
            
            # Guardar pregunta y respuesta en la lista
            data.append([current_document, current_route, current_category, question, answer])
    
    return pd.DataFrame(data, columns=["Documento", "Ruta", "Categoria_pregunta", "Pregunta", "Respuesta_Humana"])


In [28]:
# Paso 4: Extraer datos del documento de Word
df_word = extract_questions_answers(word_file)



In [29]:
# Paso 5: Cargar el archivo de Excel existente si existe
if os.path.exists(excel_file):
    df_excel = pd.read_excel(excel_file)
else:
    df_excel = pd.DataFrame(columns=["Documento", "Ruta", "Categoria_pregunta", "Pregunta", "Respuesta_Humana", "Respuesta_RAG", "Modelo_embedding_usado", "LLM_usado", "Documentos_relevantes_con_score", "Score_RHvsRRAG"])



In [30]:
# Paso 6: Combinar datos extraídos con el archivo de Excel existente
df_combined = pd.concat([df_excel, df_word], ignore_index=True)



In [31]:
# Paso 7: Guardar el archivo Excel actualizado
output_file = "corpus_extranjerIA_completo.xlsx"
df_combined.to_excel(output_file, index=False)

print(f"Archivo guardado exitosamente como {output_file}")

Archivo guardado exitosamente como corpus_extranjerIA_completo.xlsx


In [32]:
# Cargar el archivo de Excel actualizado
df_final = pd.read_excel("corpus_extranjerIA_completo.xlsx")

# Contar la cantidad de preguntas por categoría
categoria_counts = df_final["Categoria_pregunta"].value_counts()

# Mostrar los resultados
print("Cantidad de preguntas por categoría:")
print(categoria_counts)

Cantidad de preguntas por categoría:
Categoria_pregunta
Preguntas factuales          93
Preguntas inferenciales      52
Preguntas procedimentales    51
Preguntas comparativas       37
Name: count, dtype: int64


In [33]:
num_documentos_unicos = df_final["Documento"].nunique()
print(f"Número de documentos únicos en el dataframe: {num_documentos_unicos}")

Número de documentos únicos en el dataframe: 31


In [34]:
# Cargar el archivo de Excel generado
df = pd.read_excel("corpus_extranjerIA_completo.xlsx")

# Filtrar filas eliminando aquellas donde Respuesta_Humana está vacía o contiene ":"
df_cleaned = df[df["Respuesta_Humana"].notna() & ~df["Respuesta_Humana"].str.contains(":", na=False)]

# Guardar el nuevo archivo limpio
output_cleaned_file = "corpus_extranjerIA_limpio.xlsx"
df_cleaned.to_excel(output_cleaned_file, index=False)

print(f"Archivo limpio guardado como {output_cleaned_file}")


Archivo limpio guardado como corpus_extranjerIA_limpio.xlsx


In [35]:
num_documentos_unicos = df["Documento"].nunique()
print(f"Número de documentos únicos en el dataframe: {num_documentos_unicos}")

Número de documentos únicos en el dataframe: 31


In [36]:
documentos_unicos = df["Documento"].unique()
print(documentos_unicos)

['AUT_TRABJ_Familiares_UE_sin_TIE.pdf' 'Enlace1_Solicitar_duplicado.pdf'
 'Guia_informativa_basica_25-05-2022.pdf' 'Nota_empleadores.pdf'
 'Presentar_recursos.pdf' '22bis_Estudiantes-ERASMUS.pdf'
 '4_Busqueda_empleo_proyecto_empresarial.pdf'
 'Autorizacion_estancia_estudios_inicial.pdf'
 'Estudiantes-residencia_practicas.pdf' 'Estudiantes-trabajoparcial.pdf'
 'Extranjero_no_UE_estudiante_otro_pais_UE.pdf'
 'Familiares_estudiante_sin_NIE.pdf' 'Prorroga_estancia_estudios.pdf'
 '16Familiar_comunitario_inicial.pdf'
 '17Familiar_comunitario_permanente.pdf' '1Arraigo_familiar_inicial.pdf'
 '2Arraigo_social.pdf' '27Razones_humanitarias.pdf'
 '28Proteccion_internacional.pdf'
 '29bisViolencia_genero_Tarjeta_provisional.pdf' '29Violencia_genero.pdf'
 'Autorizacion_trabajo.pdf' '1_Cuenta_ajena-inicial.pdf'
 '2_cuenta_ajena-renovacion.pdf' '3-_CUENTA_AJENA-modificacion.pdf'
 '4_Cuenta_ajena-modificacion_otra_situacion.pdf'
 '5_Cuenta_propia-renovacion.pdf' '26No_lucrativa_renovacion.pdf'
 'Como_pe

In [16]:
# Cargar el archivo de Excel actualizado
df_final = pd.read_excel("corpus_extranjerIA_limpio.xlsx")

# Contar la cantidad de preguntas por categoría
categoria_counts = df_final["Categoria_pregunta"].value_counts()

# Mostrar los resultados
print("Cantidad de preguntas por categoría:")
print(categoria_counts)

Cantidad de preguntas por categoría:
Categoria_pregunta
Preguntas inferenciales      52
Preguntas factuales          51
Preguntas comparativas       28
Preguntas procedimentales    21
Name: count, dtype: int64


In [17]:
num_documentos_unicos = df_final["Documento"].nunique()
print(f"Número de documentos únicos en el dataframe: {num_documentos_unicos}")

Número de documentos únicos en el dataframe: 30


In [ ]:
documentos_unicos = combined_df["Documento"].unique()
print(documentos_unicos)

In [157]:
import numpy as np

# Fijar una semilla para hacer la selección reproducible
np.random.seed(42)

# Seleccionar 15 preguntas al azar por cada categoría
df_sampled = df_final.groupby("Categoria_pregunta").apply(lambda x: x.sample(n=10, random_state=42)).reset_index(drop=True)

# Guardar el resultado en un nuevo archivo de Excel
output_sampled_file = "corpus_extranjerIA_muestra.xlsx"
df_sampled.to_excel(output_sampled_file, index=False)

print(f"Archivo guardado exitosamente como {output_sampled_file}")

Archivo guardado exitosamente como corpus_extranjerIA_muestra.xlsx


C:\Users\zepol\AppData\Local\Temp\ipykernel_18100\2433714289.py:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sampled = df_final.groupby("Categoria_pregunta").apply(lambda x: x.sample(n=10, random_state=42)).reset_index(drop=True)


In [158]:
# Cargar el archivo de Excel actualizado
df_final = pd.read_excel("corpus_extranjerIA_muestra.xlsx")

# Contar la cantidad de preguntas por categoría
categoria_counts = df_final["Categoria_pregunta"].value_counts()

# Mostrar los resultados
print("Cantidad de preguntas por categoría:")
print(categoria_counts)

Cantidad de preguntas por categoría:
Categoria_pregunta
Preguntas comparativas       10
Preguntas factuales          10
Preguntas inferenciales      10
Preguntas procedimentales    10
Name: count, dtype: int64
